
# Shared rides with pricing - Delft 

 ### Choice Function (Deterministic):
 `pool_price.py`
 * Pickup Distance: distance from driver initial position to the first pickup point
 * Travel Distance: distance from driver's initial position to the drop off point of the last passenger
 * Operating Cost: This include all the expenses
 * Profit: Driver revenue to serve the request
            
 
  ### KPI:
   
   * Profit of Individual driver
   * Profit of all the drivers
   * No.of rejected rides
   * U - PAX (Utility) 
  
   ### TBD- Choice Function (Probablistic):
  
  * choice logic to be applied inside `pool_price.py` 
  * P(R)= exp(beta * Profit_R)/ sum_all the rides( exp(beta * Profit_R)
 



-------------------------------------------------------------------------------------------------------

# Pricing and Driver Earnings for a Two-Sided Mobility Platform: A Case of Amsterdam, the Netherlands

or 

# The Effects of Profit-Based Pricing on Driver Earnings and Performance of Two-Sided Mobility Platforms

# Abstract  

In this paper, we investigate how the  pricing of ride-pooling affects driver earnings. We also examine how profit-based setting affects these performance indicators. To this end, we applied a matching algorithm  to the case of ride-pooling and give a choice set to the driver for the case of Amsterdam, the Netherlands. For our simulation, we utilize an agent-based simulator reproducing the transport systems for two-sided mobility platforms (like Uber and Lyft) and applied three state-of-the-art pricing strategies such as <strong>profit maximization</strong>,  <strong>solo ride-hailing</strong>, and <strong>nearest pickup ride-pooling</strong>. We find that the profit maximization pricing strategy outperforms the other and traveler utility can be further improved by $\%X$ while reducing the total cost to serve the pooled rides. While offering a discount for profit maximization travel time is significantly higher $\%X$  than for private rides. 

-------------------------------------------------------------------------------------------------------

## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo ride-hailing</strong>

<strong>3. Nearest pickup ride-pooling</strong>

## Load ExMAS and MaaSSim


In [1]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

## Delft, Netherlands

In [2]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 400 # reuqests (and passengers)
params.nV = 5 # vehicles



## Parameters for ExMAS

In [3]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# Strategy 1: 
# params.kpi = 1 (Profit Maximazation)


### Profit Mazimization - Begin 

In [4]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

26-02-23 11:59:29-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
26-02-23 11:59:29-INFO-creating combinations
26-02-23 11:59:29-INFO-9900	 nR*(nR-1)
26-02-23 11:59:29-INFO-Reduction of feasible pairs by 99.21%
26-02-23 11:59:29-INFO-Degree 2 	Completed
26-02-23 11:59:29-INFO-trips to extend at degree 2 : 154
26-02-23 11:59:29-INFO-At degree 2 feasible extensions found out of 10 searched
26-02-23 11:59:29-INFO-Degree 3 	Completed
26-02-23 11:59:29-INFO-trips to extend at degree 3 : 10
26-02-23 11:59:29-INFO-At degree 3 feasible extensions found out of 0 searched
26-02-23 11:59:29-INFO-Degree 4 	Completed
26-02-23 11:59:29-INFO-Max degree reached 4
26-02-23 11:59:29-INFO-Trips still possible to extend at degree 4 : 0


In [5]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

26-02-23 11:59:30-INFO-Matching 100 trips to 264 rides in order to min u_veh
26-02-23 11:59:30-INFO-Problem solution: Optimal. 
Total costs for single trips:         25,299 
reduced by matching to:               21,503


In [6]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],7.881000,426,1,[7.881],"[0, 426]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1569647891, 4349049067]","[None, 0, 0]",node time req_id od 0 ...,426,4.26,6.390000,1.278000,5.1120
1,[1],4.535500,245,1,[4.5355],"[17, 245]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1575265415, 44855139]","[None, 1, 1]",node time req_id od 0 ...,245,2.45,3.678000,0.735600,2.9424
2,[2],2.252000,121,1,[2.252],"[31, 121]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44832711, 1413910861]","[None, 2, 2]",node time req_id od 0 ...,121,1.21,1.828500,0.365700,1.4628
3,[3],5.553000,300,1,[5.553],"[35, 300]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44829654, 6597272889]","[None, 3, 3]",node time req_id od 0 ...,300,3.00,4.503000,0.900600,3.6024
4,[4],4.890000,264,1,[4.890000000000001],"[141, 264]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44814383, 4955735201]","[None, 4, 4]",node time req_id od 0 ...,264,2.64,3.966000,0.793200,3.1728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,"[72, 66, 77]",19.993750,737,32,"[10.257850000000001, 5.4300250000000005, 4.305...","[2351.0, 78, 241, 121, 173, 124]","[72, 66, 77]","[66, 77, 72]",3,259,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1584637042, 44718822, 1699296345, 44802...","[None, 72, 66, 77, 66, 77, 72]",node time req_id od 0 ...,737,7.37,12.600000,2.520000,10.0800
260,"[22, 17, 23]",17.119725,667,31,"[8.7569, 5.1724, 3.190425]","[539.0, 30, 80, 184, 110, 263]","[22, 17, 23]","[23, 17, 22]",3,260,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 5133225964, 1448535876, 44758563, 14485...","[None, 22, 17, 23, 23, 17, 22]",node time req_id od 0 ...,667,6.67,10.717875,2.143575,8.5743
261,"[44, 43, 46]",22.464200,869,30,"[10.07725, 5.80245, 6.5845]","[1202.0, 94, 262, 178, 262, 73]","[44, 43, 46]","[43, 44, 46]",3,261,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44853267, 44875062, 1436427166, 8393352...","[None, 44, 43, 46, 43, 44, 46]",node time req_id od 0 ...,869,8.69,14.460750,2.892150,11.5686
262,"[44, 37, 49]",20.389150,830,32,"[10.443525000000001, 4.78435, 5.161275]","[1103.5, 186, 89, 256, 130, 169]","[44, 37, 49]","[37, 49, 44]",3,262,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44853267, 3905428979, 44866496, 1552650...","[None, 44, 37, 49, 37, 49, 44]",node time req_id od 0 ...,830,8.30,12.742875,2.548575,10.1943


In [43]:
params.kpi = 1

In [ ]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

26-02-23 10:39:08-INFO-Matching 300 trips to 699 rides in order to min u_veh
26-02-23 10:39:08-INFO-Problem solution: Optimal. 
Total costs for single trips:         78,882 
reduced by matching to:               68,407
0.5
0.5
0.5
26-02-23 10:39:13-CRITICAL-this is request 2 with [2, 336, 342, 349, 357, 383, 416, 488, 549, 550, 551, 552, 553, 686, 687, 693] available rides.
26-02-23 10:39:13-CRITICAL-ride 2 available [1]
26-02-23 10:39:13-CRITICAL-ride 336 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 342 available [0, 1]
26-02-23 10:39:13-CRITICAL-ride 349 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 357 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 383 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 416 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 488 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 549 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 550 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 551 available [1, 0]
26-02-23 10:39:13-CRITICAL-ride 552 available

In [ ]:
sim.res[0].veh_exp

In [ ]:
sim.res[0].veh_exp['REVENUE'].to_list()

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")
sim.res[0].veh_exp['Vehicles'] = sim.res[0].veh_exp.index

ax =sns.barplot(data=sim.res[0].veh_exp, x="Vehicles", y="REVENUE")
#for i in ax.containers:
    #ax.bar_label(i,)

# Total Revenue of all the driver 

In [ ]:
sim.res[0].all_kpi # All driver revenue 

# Strategy 2: 

# params.kpi = 2 (Pooled Ride - Nearest) 


In [ ]:
params.kpi = 2

In [ ]:
sim = simulate(params = params, inData = inData, logger_level = logging.WARNING) # simulate

In [ ]:
sim.res[0].veh_exp

In [ ]:
sim.res[0].veh_exp['REVENUE'].to_list()

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")

sim.res[0].veh_exp['Vehicles'] = sim.res[0].veh_exp.index

ax =sns.barplot(data=sim.res[0].veh_exp, x="Vehicles", y="REVENUE")
#for i in ax.containers:
    #ax.bar_label(i,)

# Total revenue of all the driver

In [ ]:
sim.res[0].all_kpi # All driver revenue 

# Strategy 3: 
# params.kpi = 3 (Private ride)


In [ ]:
params.kpi = 3

In [ ]:
sim = simulate(params = params, inData = inData, logger_level = logging.WARNING) # simulate

In [ ]:
sim.res[0].veh_exp

In [ ]:
sim.res[0].veh_exp['REVENUE'].to_list()

In [ ]:
import seaborn as sns

sns.set_style("whitegrid")

sim.res[0].veh_exp['Vehicles'] = sim.res[0].veh_exp.index

ax =sns.barplot(data=sim.res[0].veh_exp, x="Vehicles", y="REVENUE")

#ax.set(xlabel=None)
#for i in ax.containers:
    #ax.bar_label(i,)

# Total revenue of all the driver 

In [16]:
sim.res[0].all_kpi # All driver revenue 

58.006800000000005

# All in one Simulation  

In [7]:
responses = []
avg_kpi = []
idle_time = []

for i in range(1, 4):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)
    #['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
   # responses.append(sim.res[0].veh_exp)
    
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
   #no_of_veh = len(vehicles)
    
   #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
   # idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    

26-02-23 11:59:35-INFO-Matching 100 trips to 264 rides in order to min u_veh
26-02-23 11:59:35-INFO-Problem solution: Optimal. 
Total costs for single trips:         25,299 
reduced by matching to:               21,503
0.5
26-02-23 11:59:37-CRITICAL-this is request 0 with [0, 166, 167, 168, 169, 170, 171, 246, 247] available rides.
26-02-23 11:59:37-CRITICAL-ride 0 available [1]
26-02-23 11:59:37-CRITICAL-ride 166 available [1, 0]
26-02-23 11:59:37-CRITICAL-ride 167 available [1, 0]
26-02-23 11:59:37-CRITICAL-ride 168 available [1, 0]
26-02-23 11:59:37-CRITICAL-ride 169 available [1, 0]
26-02-23 11:59:37-CRITICAL-ride 170 available [1, 0]
26-02-23 11:59:37-CRITICAL-ride 171 available [1, 0]
26-02-23 11:59:37-CRITICAL-ride 246 available [1, 0]
26-02-23 11:59:37-CRITICAL-ride 247 available [1, 0]
26-02-23 11:59:37-CRITICAL-this is reuqest 0 with [0, 166, 167, 168, 169, 170, 171, 246, 247] still available rides.
Profit Maximization
26-02-23 11:59:37-CRITICAL-vehicle 1 has 9 choices
0.5
26

# Performance Parameters for Driver

In [8]:
import pandas as pd
index = pd.Index(['Revenue', 'Profit', 'Cost', 'Idle Time'])
driver_data = pd.DataFrame({"Profit Maximization":[], "Pooled Ride": [], "Private Ride": []})
driver_data.loc['Revenue'] = avg_kpi
driver_data.loc['Idle Time'] = idle_time
driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)

In [9]:
driver_data

,Profit Maximization,Pooled Ride,Private Ride
Revenue,4.192905,4.192905,4.192905
Idle Time,3566.400000,3542.550000,3514.450000
Cost,2.096452,2.096452,2.096453


In [ ]:
csv_data = driver_data.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/nV20.csv')

In [ ]:
print('\nCSV String:\n', csv_data)